In [38]:
import pandas as pd
import pickle

path = 'data/gamecodes.data'
with open(path, 'rb') as f:
    gcodes = pickle.load(f)

In [82]:
teams = {'Atlanta Falcons':'atl','Buffalo Bills':'buf','Carolina Panthers':'car','Chicago Bears':'chi',
         'Cincinnati Bengals':'cin','Cleveland Browns':'cle','Indianapolis Colts':'clt',
         'Arizona Cardinals':'crd','Dallas Cowboys':'dal','Denver Broncos':'den','Detroit Lions':'det',
         'Green Bay Packers':'gnb','Houston Texans':'htx','Jacksonville Jaguars':'jax',
         'Kansas City Chiefs':'kan','Miami Dolphins':'mia','Minnesota Vikings':'min','New Orleans Saints':'nor',
         'New England Patriots':'nwe','New York Giants':'nyg','New York Jets':'nyj','Tennessee Titans':'oti',
         'Philadelphia Eagles':'phi','Pittsburgh Steelers':'pit','Oakland Raiders':'rai',
         'Las Vegas Raiders':'rai','St. Louis Rams':'ram','Los Angeles Rams':'ram','Baltimore Ravens':'rav',
         'San Diego Chargers':'sdg','Los Angeles Chargers':'sdg','Seattle Seahawks':'sea',
         'San Francisco 49ers':'sfo','Tampa Bay Buccaneers':'tam','Washington Redskins':'was'}
cols = ['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','V_Off_Pass','H_Off_Rush','V_Off_Rush',
        'H_Def_Pass','V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def','H_FG_Made',
        'V_FG_Made','H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv',
        'V_RZ_Def_Conv','H_RZ_Def_Att','V_RZ_Def_Att','H_Poss','V_Poss','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain',
        'H_TO_Lost','V_TO_Lost','H_Yds_Pen','V_Yds_Pen','H_Sacks_Def','V_Sacks_Def','H_Tackles_Loss','V_Tackles_Loss',
        'H_Kickret','V_Kickret','H_Puntret','V_Puntret']
hdictmap = {'Points':'H_Pts','Points_Opp':'H_Pts_Opp','Yds_Off_Pass':'H_Off_Pass','Yds_Off_Rush':'H_Off_Rush',
           'Yds_Def_Pass':'H_Def_Pass','Yds_Def_Rush':'H_Def_Rush','TD':'H_TD','TD_on_Def':'H_TD_on_Def',
            'FG_Made':'H_FG_Made','FG_Att':'H_FG_Att','RZ_Conv':'H_RZ_Conv','RZ_Att':'H_RZ_Att',
            'RZ_Def_Conv':'H_RZ_Def_Conv','RZ_Def_Att':'H_RZ_Def_Att','Possession':'H_Poss','Plays':'H_Plays',
           'TO_Gained':'H_TO_Gain','TO_Lost':'H_TO_Lost','Yds_Pen':'H_Yds_Pen','Sacks_Def':'H_Sacks_Def',
           'Tackles_Loss':'H_Tackles_Loss','Yds_per_Kickret':'H_Kickret','Yds_per_Puntret':'H_Puntret'}
vdictmap = {'Points':'V_Pts','Points_Opp':'V_Pts_Opp','Yds_Off_Pass':'V_Off_Pass','Yds_Off_Rush':'V_Off_Rush',
           'Yds_Def_Pass':'V_Def_Pass','Yds_Def_Rush':'V_Def_Rush','TD':'V_TD','TD_on_Def':'V_TD_on_Def',
            'FG_Made':'V_FG_Made','FG_Att':'V_FG_Att','RZ_Conv':'V_RZ_Conv','RZ_Att':'V_RZ_Att',
            'RZ_Def_Conv':'V_RZ_Def_Conv','RZ_Def_Att':'V_RZ_Def_Att','Possession':'V_Poss','Plays':'V_Plays',
           'TO_Gained':'V_TO_Gain','TO_Lost':'V_TO_Lost','Yds_Pen':'V_Yds_Pen','Sacks_Def':'V_Sacks_Def',
           'Tackles_Loss':'V_Tackles_Loss','Yds_per_Kickret':'V_Kickret','Yds_per_Puntret':'V_Puntret'}

In [89]:
years = [str(year) for year in range(2010,2020)]
years = [years[0]]
#codes = [gcodes[i] for i in range(len(gcodes)) 
#      if (((gcodes[i][0:4] == year) & (gcodes[i][4:6] != '01')) | (gcodes[i][0:6] == (str(int(year)+1)+'01')))]

In [84]:
dfgame=pd.DataFrame(index=gcodes,columns=cols)
dfgame.index.set_names('Code',inplace=True)

In [91]:
for year in years:
    path = 'data/df_step2_'+year+'.data'
    with open(path, 'rb') as f:
        dfyear = pickle.load(f)
    codes = set([ind[1] for ind in dfyear.index])
    for code in codes:
        home = code[-3:]
        away = teams[dfyear.loc[(home,code),'Opponent']]
        for key in hdictmap:
            dfgame.loc[code,hdictmap[key]] = dfyear.loc[(home,code),key]
        for key in vdictmap:
            dfgame.loc[code,vdictmap[key]] = dfyear.loc[(away,code),key]

In [106]:
for team, dfteam in dfyear.groupby(level=0):
    print(dfteam)

                  Home/Away              Opponent  Points  Points_Opp  \
Team Code                                                               
atl  201009120pit         A   Pittsburgh Steelers       0           6   
     201009190atl         H     Arizona Cardinals      41           7   
     201009260nor         A    New Orleans Saints       3           0   
     201010030atl         H   San Francisco 49ers      16          14   
     201010100cle         A      Cleveland Browns      20          10   
     201010170phi         A   Philadelphia Eagles      17          31   
     201010240atl         H    Cincinnati Bengals      39          32   
     201011070atl         H  Tampa Bay Buccaneers      27          21   
     201011110atl         H      Baltimore Ravens      26          21   
     201011210ram         A        St. Louis Rams      34          17   
     201011280atl         H     Green Bay Packers      20          17   
     201012050tam         A  Tampa Bay Buccaneers  

                  Home/Away             Opponent  Points  Points_Opp  \
Team Code                                                              
jax  201009120jax         H       Denver Broncos      24          17   
     201009190sdg         A   San Diego Chargers      13          38   
     201009260jax         H  Philadelphia Eagles       3          28   
     201010030jax         H   Indianapolis Colts      31          28   
     201010100buf         A        Buffalo Bills      36          26   
     201010180jax         H     Tennessee Titans       3          30   
     201010240kan         A   Kansas City Chiefs      20          42   
     201010310dal         A       Dallas Cowboys      35          17   
     201011140jax         H       Houston Texans      31          24   
     201011210jax         H     Cleveland Browns      24          20   
     201011280nyg         A      New York Giants      20          24   
     201012050oti         A     Tennessee Titans      17        

                  Home/Away              Opponent  Points  Points_Opp  \
Team Code                                                               
sdg  201009130kan         A    Kansas City Chiefs      14          21   
     201009190sdg         H  Jacksonville Jaguars      38          13   
     201009260sea         A      Seattle Seahawks      20          27   
     201010030sdg         H     Arizona Cardinals      41          10   
     201010100rai         A       Oakland Raiders      27          35   
     201010170ram         A        St. Louis Rams      17          20   
     201010240sdg         H  New England Patriots      20          23   
     201010310sdg         H      Tennessee Titans      33          25   
     201011070htx         A        Houston Texans      29          23   
     201011220sdg         H        Denver Broncos      35          14   
     201011280clt         A    Indianapolis Colts      36          14   
     201012050sdg         H       Oakland Raiders  

In [42]:
dfyear

Home/Away              Opponent  Points  Points_Opp  \
Team Code                                                               
phi  201009120phi         H     Green Bay Packers      20          27   
     201009190det         A         Detroit Lions      35          32   
     201009260jax         A  Jacksonville Jaguars      28           3   
     201010030phi         H   Washington Redskins      12          17   
     201010100sfo         A   San Francisco 49ers      27          24   
...                     ...                   ...     ...         ...   
car  201012050sea         A      Seattle Seahawks      14          31   
     201012120car         H       Atlanta Falcons      10          31   
     201012190car         H     Arizona Cardinals      19          12   
     201012230pit         A   Pittsburgh Steelers       3          27   
     201101020atl         A       Atlanta Falcons      10          31   

                   Yds_Off_Pass  Yds_Off_Rush  Yds_Def_Pass  Yds_Def_Rush  TD  \
Team Code                                                                       
phi  201009120phi           199           149           188           132   2   
     201009190det           284           162           335           115   5   
     201009260jax           291           106           105           139   4   
     201010030phi           250           110           125           169   1   
     201010100sfo           253           125           309            74   3   
...                         ...           ...           ...           ...  ..   
car  201012050sea           169           131           229           161   2   
     201012120car           107           212           227           127   1   
     201012190car           141           177           196            43   1   
     201012230pit            72            74           320           115   0   
     201101020atl           182           137           256            99   2   

                   TD_on_Def  ...  RZ_Def_Att  Possession  Plays  TO_Gained  \
Team Code                     ...                                             
phi  201009120phi          0  ...           2   28.083333     62          2   
     201009190det          0  ...           4   30.866667     68          2   
     201009260jax          0  ...           2   27.116667     55          1   
     201010030phi          0  ...           2   32.950000     70          1   
     201010100sfo          1  ...           3   30.800000     65          5   
...                      ...  ...         ...         ...    ...        ...   
car  201012050sea          0  ...           3   30.500000     67          2   
     201012120car          0  ...           5   25.166667     59          1   
     201012190car          0  ...           3   36.933333     70          2   
     201012230pit          0  ...           6   24.600000     50          2   
     201101020atl          0  ...           7   24.500000     57          1   

                   TO_Lost  Yds_Pen  Sacks_Def  Tackles_Loss  Yds_per_Kickret  \
Team Code                                                                       
phi  201009120phi        1       80        3.0             6        23.250000   
     201009190det        0       75        2.0            10        21.800000   
     201009260jax        0       84        7.0             2        16.750000   
     201010030phi        2       80        1.0             2        12.000000   
     201010100sfo        1       30        2.0             5        25.200000   
...                    ...      ...        ...           ...              ...   
car  201012050sea        1       53        3.0             5        21.666667   
     201012120car        2       50        3.0             5        17.666667   
     201012190car        1       58        3.0             5        17.500000   
     201012230pit        2       62        3.0             6        22.000000   
     201101020

In [98]:
dfgame.head(40)

,H_Pts,V_Pts,H_Pts_Opp,V_Pts_Opp,H_Off_Pass,V_Off_Pass,H_Off_Rush,V_Off_Rush,H_Def_Pass,V_Def_Pass,...,H_Yds_Pen,V_Yds_Pen,H_Sacks_Def,V_Sacks_Def,H_Tackles_Loss,V_Tackles_Loss,H_Kickret,V_Kickret,H_Puntret,V_Puntret
Code,,,,,,,,,,,,,,,,,,,,,
201009120phi,20,27,27,20,199,188,149,132,188,199,...,80,15,3,6,6,5,23.25,31.2,7,10
201009190det,32,35,35,32,335,284,115,162,284,335,...,51,75,5,2,7,10,16.5,21.8,5,11.3333
201009260jax,3,28,28,3,105,291,139,106,291,105,...,67,84,3,7,12,2,29.5,16.75,10.3333,12.4
201010030phi,12,17,17,12,250,125,110,169,125,250,...,80,30,1,1,2,5,12,23,25,10
201010100sfo,24,27,27,24,309,253,74,125,253,309,...,65,30,4,2,4,5,36.2,25.2,4.5,11.5
201010170phi,31,17,17,31,326,250,154,65,250,326,...,78,30,3,1,4,8,9.5,23.3333,16.6667,18
201010240oti,37,19,19,37,276,231,75,99,231,276,...,37,100,1,3,2,5,3,31.5,10.75,8
201011070phi,26,24,24,26,218,294,195,62,294,218,...,125,59,3,3,5,6,18.75,14.5,2.5,0
201011150was,28,59,59,28,295,333,105,260,333,295,...,41,35,1,2,2,5,24.1429,16.5,6,9


In [97]:
dfgame.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2560 entries, 201009120phi to 201910270ram
Data columns (total 46 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   H_Pts           256 non-null    object
 1   V_Pts           256 non-null    object
 2   H_Pts_Opp       256 non-null    object
 3   V_Pts_Opp       256 non-null    object
 4   H_Off_Pass      256 non-null    object
 5   V_Off_Pass      256 non-null    object
 6   H_Off_Rush      256 non-null    object
 7   V_Off_Rush      256 non-null    object
 8   H_Def_Pass      256 non-null    object
 9   V_Def_Pass      256 non-null    object
 10  H_Def_Rush      256 non-null    object
 11  V_Def_Rush      256 non-null    object
 12  H_TD            256 non-null    object
 13  V_TD            256 non-null    object
 14  H_TD_on_Def     256 non-null    object
 15  V_TD_on_Def     256 non-null    object
 16  H_FG_Made       256 non-null    object
 17  V_FG_Made       256 non-null    object

In [ ]:
cols = ['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','V_Off_Pass','H_Off_Rush','V_Off_Rush',
        'H_Def_Pass','V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def','H_FG_Made',
        'V_FG_Made','H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv',
        'V_RZ_Def_Conv','H_RZ_Def_Att','V_RZ_Def_Att','H_Poss','V_Poss','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain',
        'H_TO_Lost','V_TO_Lost','H_Yds_Pen','V_Yds_Pen','H_Sacks_Def','V_Sacks_Def','H_Tackles_Loss','V_Tackles_Loss',
        'H_Kickret','V_Kickret','H_Puntret','V_Puntret']

In [96]:
intcols=['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','V_Off_Pass','H_Off_Rush','V_Off_Rush','H_Def_Pass',
        'V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def','H_FG_Made','V_FG_Made',
        'H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv','V_RZ_Def_Conv',
        'H_RZ_Def_Att','V_RZ_Def_Att','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain','H_TO_Lost','V_TO_Lost',
        'H_Yds_Pen','V_Yds_Pen','H_Tackles_Loss','V_Tackles_Loss']
fltcols=['H_Poss','V_Poss','H_Sacks_Def','V_Sacks_Def','H_Kickret','V_Kickret','H_Puntret','V_Puntret']
for colu in intcols: dfgame[colu] = dfgame[colu].astype('int64')
for colu in fltcols: dfgame[colu] = dfgame[colu].astype('float64')

ValueError: cannot convert float NaN to integer

In [43]:
dfyear.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 512 entries, ('phi', '201009120phi') to ('car', '201101020atl')
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Home/Away        512 non-null    object 
 1   Opponent         512 non-null    object 
 2   Points           512 non-null    int64  
 3   Points_Opp       512 non-null    int64  
 4   Yds_Off_Pass     512 non-null    int64  
 5   Yds_Off_Rush     512 non-null    int64  
 6   Yds_Def_Pass     512 non-null    int64  
 7   Yds_Def_Rush     512 non-null    int64  
 8   TD               512 non-null    int64  
 9   TD_on_Def        512 non-null    int64  
 10  FG_Made          512 non-null    int64  
 11  FG_Att           512 non-null    int64  
 12  RZ_Conv          512 non-null    int64  
 13  RZ_Att           512 non-null    int64  
 14  RZ_Def_Conv      512 non-null    int64  
 15  RZ_Def_Att       512 non-null    int64  
 16  Possession       512